In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os,glob
import time 


## Acessando o site CVM

In [ ]:
driver = webdriver.Chrome('C:/Users/Thiago Jose/Downloads/chromedriver-win64/chromedriver-win64/chromedriver')

url = '''https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx'''
driver.implicitly_wait(10) #PARA ESPERAR O CARREGAMENTO DO SITE
driver.get(url)
driver.maximize_window()

### Selecionando empresas abertas

In [88]:
selecting_open_companies = driver.find_element('xpath','/html/body/form[1]/div[3]/div/fieldset/div[3]/div[1]/div/div/ul/li[2]/input')

selecting_open_companies.send_keys("Companhia Aberta")

selecting_open_companies.send_keys(Keys.RETURN)

driver.implicitly_wait(10) # waiting

#### MARCANDO O PERÍODO NO CHECKBOX

In [89]:
checkbox_periodo = driver.find_element('xpath','/html/body/form[1]/div[3]/div/fieldset/div[4]/div[1]/div[1]/div[3]/input')
driver.implicitly_wait(3) #PARA ESPERAR O CARREGAMENTO DO SITE
driver.execute_script("arguments[0].click();",checkbox_periodo)

##### Atribuindo datas início e fim

In [92]:

#Data de inicio
driver.implicitly_wait(10) #PARA ESPERAR O CARREGAMENTO DO SITE
input_data_inicio = driver.find_element('xpath', '/html/body/form[1]/div[3]/div/fieldset/div[4]/div[1]/div[2]/div[1]/input[1]')
input_data_inicio.send_keys("01012012")

#Data de fim
driver.implicitly_wait(10) #PARA ESPERAR O CARREGAMENTO DO SITE
input_data_fim = driver.find_element('xpath', '/html/body/form[1]/div[3]/div/fieldset/div[4]/div[1]/div[2]/div[2]/input[1]')
input_data_fim.send_keys("31032016")



input_data_fim.send_keys(Keys.RETURN)  # Aperta tecla enter 


#### Selecionando o press-release e apertando em consultar

In [90]:
select_press_release = driver.find_element('xpath','/html/body/form[1]/div[3]/div/fieldset/div[5]/div[1]/div[2]/div/div/div/input')

select_press_release.send_keys("Press-release")

select_press_release.click()
select_press_release.send_keys(Keys.RETURN)


In [91]:
#Selecionando lista da tabela ocultar para apenas documentos
hide_about = driver.find_element('xpath','/html/body/form[1]/div[5]/div/div[2]/label/table/tbody/tr/td[1]/i[2]')

driver.execute_script('arguments[0].click();',hide_about)

In [8]:
# #Apertando botão de consulta
time.sleep(5)
search_btn = driver.find_element('xpath','/html/body/form[1]/div[3]/div/fieldset/div[7]/div/input')
driver.execute_script("arguments[0].click();",search_btn)

#### Download Press-release

In [4]:
## Função que muda o nome e leva o arquivo para a pasta pre-definida

def get_last_filename_and_rename(save_folder, new_filename):
    time.sleep(2)
    
   
    new_filename = new_filename.replace('/','-')
   
    files = glob.glob(save_folder + '\*') # Lista todos os arquivos do download
  
    max_file = max(files, key=os.path.getctime) # seleciona o com maior data-hora de modificacao
    # print(max_file)
    # while not os.path.exists(file_path):
    time.sleep(1)

    
    filename = max_file.split("/")[-1].split(".")[0] # separa o caminho do nome do arquivo e a extensao
    
    new_path = max_file.replace(filename, new_filename) # substitui o nome do arquivo com o novo nome
    new_path = new_path.split("/")[-1] # tira o ultimo resquicio do path do arquivo
    new_path = 'C:/Users/Thiago Jose/Desktop/Press-Release/'+ new_path # atribui um novo caminho

    # print(new_path)
    os.rename(max_file, new_path) # muda o arquivo para o local selecionado com o novo nome
    return new_path


# get_last_filename_and_rename('C:/Users/Thiago Jose/Downloads','teste2')

In [59]:
# def get_last_filename_and_rename(save_folder, new_filename):
#     files = glob.glob(save_folder + '\*')
#     print(files)
#     max_file = max(files, key=os.path.getctime)
#     print(max_file)
#     filename = max_file.split("/")[-1].split(".")[0]
#     print(filename)
#     new_path = max_file.replace(filename, new_filename)
#     new_path = new_path.split("/")[-1]
#     new_path = 'C:/Users/Thiago Jose/Desktop/Press-Release/'+ new_path
#     print("-->", new_path)
#     os.rename(max_file, new_path)
#     return new_path


# get_last_filename_and_rename('C:/Users/Thiago Jose/Downloads','testsaae21')

In [ ]:
"""  !!!!!!!  Não executar essa célula, os realeses já foram baixados e serão disponibilizados no driver:        """

lista_tabelas = [] #Vai armazenar as tabelas de cada pagina 
files_path = []
os.chdir(r"C:\Users\Thiago Jose\Downloads")

next_btn = driver.find_element('xpath','/html/body/form[1]/div[5]/div/div[4]/a[2]') #botao de proxima página
page_number = driver.find_element('xpath','/html/body/form[1]/div[5]/div/div[3]') # pega o numero de registros acumulados por página, quando chegar em 15.801 é a ultima página
page_number = page_number.text.split(" ")[2] # retorna o texto e então pegamos o numero apenas

while page_number != '3,901': #vai repetir o lanço enquanto a o numero de registros acumulados for menor que 3,901

    driver.implicitly_wait(10)
    
    page_number = driver.find_element('xpath','/html/body/form[1]/div[5]/div/div[3]')
    page_number = page_number.text.split(" ")[2]
    print('Robo estar na página:',page_number)

    table_data = driver.find_element('xpath','/html/body/form[1]/div[5]/div/table') #pegando a tabela da página atual

    html_table = table_data.get_attribute('outerHTML') # extraindo HTML

    tabela = pd.read_html(html_table)[0] #pegando a primeira tabela do HTML

    tabela = pd.DataFrame(tabela) #transformando em dataframe

    
    tabela['Data Referência'] = tabela['Data Referência'].apply(lambda x: x.split(" ")[-1]) #tira as horas da data de referencia 
    tabela['Data Entrega'] = tabela['Data Entrega'].apply(lambda x: x.split(" ")[1]) # tira as horas da data de entrega

    lista_tabelas.append(tabela) #aplica a tabela na lista de tabelas


    all_download_i = driver.find_elements(By.CLASS_NAME,'fi-download') #lista todos os <i> da tabela que tem classe download


    for i in range(len(all_download_i)):
        try:
            driver.execute_script("arguments[0].click();",all_download_i[i]) #clica em cada <i> de download da página
            
            file_new_name = tabela['Empresa'][i] +'_'+ tabela['Data Referência'][i]+'_'+str(tabela['V'][i]) #nome de salvamento do arquivo será 'nome da empresa_data referencia'
                            # 'POSITIVO TECNOLOGIA S.A._05/09/2012_-'
            time.sleep(5)
            new_path = get_last_filename_and_rename('C:/Users/Thiago Jose/Downloads',file_new_name) #salva arquivo na pasta release
            
            time.sleep(5)
            files_path.append(new_path)
        except Exception:
            pass
        #fim do for


    #espera 10s para passar de página
    time.sleep(10)
    #clica no botao para avançar de página
    try:   
        driver.execute_script("arguments[0].click();", next_btn)   
    except Exception:
        btn_id = driver.find_element(By.ID,'grdDocumentos_next')
        driver.execute_script("arguments[0].click();", btn_id)
    
#Não executar essa célula, os realeses já foram baixados e serão disponibilizados no driver:

### Salvando as tabelas e os caminhos dos arquivos para serem utilizados posteriormente 

In [37]:
df_lista_tabelas = pd.concat(lista_tabelas)
print(df_lista_tabelas.shape)

df_lista_tabelas.to_csv(r'C:\Users\Thiago Jose\Desktop\CODIGO.PY\Aulas-curso\ITAU_QUANT_DESAFIO\Database\tabelas_releases_cvm.csv')


(3953, 11)


### Lendo os arquivos salvos 

In [4]:
lista_path_file = open(r"C:\Users\Thiago Jose\Desktop\CODIGO.PY\Aulas-curso\ITAU_QUANT_DESAFIO\Database\releases_path_list.txt","r")

lista_path_file_recive=[]
for line in lista_path_file:
    print(line)
    lista_path_file_recive.append(line) 

# print(lista_path_file_recive)

In [3]:
from os import listdir
from os.path import isfile, join


path = r'C:\Users\Thiago Jose\Desktop\Press-Release'
files = []
files = [path+'/'+ f for f in listdir(path) if isfile(join(path, f))]

    
print(files)

['C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE ENERGIA SA_23-02-2016_1.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE ENERGIA SA_23-02-2016_2.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE ENERGIA SA_23-02-2016_3.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_03-08-2012_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_06-11-2012_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_06-11-2013_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_25-02-2014_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_30-06-2013_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_30-06-2014_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_30-06-2015_1.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_30-09-2012_-.pdf', 'C:\\Users\\Thiago Jose\\Desktop\\Press-Release/AES TIETE SA_30-09-2014_-.pdf', 'C:\\Users\\Thi

In [5]:
from pypdf import PdfReader
pdf = PdfReader(f'{files[0]}')
text = ""

for page in pdf.pages:
    text += page.extract_text()

    

In [7]:
dtframes = pd.read_csv(r'C:\Users\Thiago Jose\Desktop\CODIGO.PY\Aulas-curso\ITAU_QUANT_DESAFIO\Database\tabelas_releases_cvm.csv')
dtframes.drop(['Unnamed: 0'],axis=1, inplace=True)
dtframes.to_csv(r'C:\Users\Thiago Jose\Desktop\CODIGO.PY\Aulas-curso\ITAU_QUANT_DESAFIO\Database\tabelas_releases_cvm.csv')

dtframes.head(5)

In [9]:
dtframes['name_pdf'] = 0
for i in dtframes.index:
    dtframes['name_pdf'][i] = dtframes['Empresa'][i]+'_'+dtframes['Data Referência'][i] +'_'+str(dtframes['V'][i])+'.pdf'



C:\Users\Thiago Jose\AppData\Local\Temp\ipykernel_26748\1713932492.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtframes['name_pdf'][i] = dtframes['Empresa'][i]+'_'+dtframes['Data Referência'][i] +'_'+str(dtframes['V'][i])+'.pdf'


## EXTRAINDO TEXTO DOS RELEASES


In [101]:
dtframes['release_text'] = 0
dtframes.head(5)

,Código CVM,Empresa,Categoria,Tipo,Espécie,Data Referência,Data Entrega,Status,V,Modalidade,name_pdf,release_text
0,02130-0,ALIANSCE SHOPPING CENTERS S/A,Dados Econômico-Financeiros,Press-release,Earnings Release - 4T15 e 2015 -,31/12/2015,31/03/2016,Ativo,2,RE,ALIANSCE SHOPPING CENTERS S/A_31/12/2015_2.pdf,0
1,02306-0,TRIÂNGULO DO SOL AUTO-ESTRADAS S.A.,Dados Econômico-Financeiros,Press-release,Divulgação de Resultado 2015 -,31/12/2015,31/03/2016,Ativo,1,AP,TRIÂNGULO DO SOL AUTO-ESTRADAS S.A._31/12/2015...,0
2,02316-7,RODOVIAS DAS COLINAS S.A.,Dados Econômico-Financeiros,Press-release,Divulgação de Resultados 2015 -,31/12/2015,31/03/2016,Ativo,1,AP,RODOVIAS DAS COLINAS S.A._31/12/2015_1.pdf,0
3,01791-4,MMX MINERACAO E METALICOS S.A. - EM RECUPERAÇÃ...,Dados Econômico-Financeiros,Press-release,Resultados referentes ao ano de 2015 -,31/12/2016,31/03/2016,Cancelado,1,AP,MMX MINERACAO E METALICOS S.A. - EM RECUPERAÇÃ...,0
4,00243-7,CENTRAIS ELET BRAS S.A. - ELETROBRAS,Dados Econômico-Financeiros,Press-release,Informe aos Investidores 4T15 - Anexo -,31/12/2015,30/03/2016,Ativo,1,AP,CENTRAIS ELET BRAS S.A. - ELETROBRAS_31/12/201...,0


In [99]:
def find_all(name, path): #Função que acha o arquivo
    name = name.replace('S/A','S.A').replace('/','-')
    for root, dirs, files in os.walk(path):
        return os.path.join(root, name)


def extract_text(pdf_file): #função que ler o arquivo e extrai o texto
    print(pdf_file)
    pdf_file = pdf_file.replace('[','').replace(']','').replace('S/A','S.A')
    try:
        pdf = PdfReader(f'{pdf_file}')
        text = ""
    except Exception:
        pdf_file = pdf_file.replace('S.A','S-A')
        pdf = PdfReader(f'{pdf_file}')
        text = ""
    print('working on: ', pdf_file)
    for page in pdf.pages:
        try:
            # print(page)
            text += page.extract_text()
        except Exception:
            pass
    return text

In [ ]:
for i in range(len(dtframes['name_pdf'])):
     pdf_file = find_all(dtframes['name_pdf'][i],"C:\\Users\\Thiago Jose\\Desktop\\Press-Release")
     dtframes['release_text'][i] = extract_text(pdf_file)

dtframes.to_parquet(r'C:\Users\Thiago Jose\Desktop\CODIGO.PY\Aulas-curso\ITAU_QUANT_DESAFIO\Database\text_releases_cvm.parquet')
dtframes.head()
